Import packages needed

In [1]:
import suite2p
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from natsort import natsorted
from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path
import gc
gc.collect()
import importlib
import sys
from pathlib import Path
# Add the directory containing your script to the Python path
sys.path.append(r'C:\Users\User\Project-SLMonlineControl\PYsubfun')
#sys.path.append(r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\PYsubfun')

# Now you can import your script as a module
import FastBin_Suite2p as FBS

In [2]:
#importlib.reload(FBS)
# Pre-set parameters, and load the only bin files in the pre-set path folde
ConfigFolder=r'C:\Users\User\Project-SLMonlineControl\config'
#ConfigFolder=r'C:\Users\zhangl33\Projects\Project-SLMonlineControl\config'
ops0, confSet=FBS.configLoad(ConfigFolder,'SLMsetting.yml','ops3Planes.npy')
SaveFolder=FBS.suite2pInitiate(ops0,ConfigFolder)
binFile = glob.glob(ops0['save_path0'] + '/TSeries*-001.bin')    ##<<<<<<<<<<<<<<<<<<<<<<<<<< Noted that there supposed be only one Bin Data file
#Load Data
print(binFile)
rawBin, FramePerPlane, TotalFrameNeed=FBS.LoadBin(binFile,ops0)
rawBin.shape

Processed data would be saved inE:\LuSLMOnlineTest\SL0840-Ai203\02192025\suite2p
File found: C:\Users\User\Project-SLMonlineControl\config\SLMsetting.yml
Copying to: E:\LuSLMOnlineTest\SL0840-Ai203\02192025\CurrentSLMsetting.yml
Current SLMsetting copied successfully!
Folder already exists.
['E:\\LuSLMOnlineTest\\SL0840-Ai203\\02192025\\TSeries-02192025-0926-001.bin']


(24600, 512, 512)

In [8]:
ops0['spatial_scale']=2
ops0['threshold_scaling']=1
ops0['denoise']=1

In [4]:
confSet

{'save_path0': 'E:\\LuSLMOnlineTest\\SL0840-Ai203\\02192025\\',
 'SLM_Pixels_X': 512,
 'SLM_Pixels_Y': 512,
 'scan_Z': 190.72,
 'ETL': '-50 0 50',
 'IsSpiral': 'True',
 'SpiralSizeUM': 15,
 'SpiralRevolution': 3,
 'Repetition': 5,
 'Duration': 20,
 'InterPointDelay': 30,
 'UncagingLaserPower': '1.45 1.55 1.65',
 'InitialDelay': 22.1,
 'NumNonTarget': 20,
 'NumNTperTrial': 9,
 'NumTrial': 5,
 'RadiusAvoidParam': 3,
 'nchannels': 1,
 'fs': 6.9,
 'tau': 0.9,
 'Ziteration': 11,
 'ZRepetition': 34}

Suite2p processing with no motion correction; Noted following defining a function to process a plane of data; Next session, parallel processing for each plane is excuted. Much time is saved. 
Alternatively, run this process with Step2-OnlineBinSuite2p.ipynb

In [9]:
# Define the number of parallel processes; 
# Noted that current binning movie in chuns of length 06; increase this parameter could decrease the number of frames within a chunks, using less memory
num_processes=1              ## <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Create a list of plane indices
plane_indices = range(0,ops0['nplanes'])
print(plane_indices)
ops1=ops0
Parallel(n_jobs=num_processes)(delayed(FBS.process_plane)(plane_idx, rawBin, SaveFolder, ops1) for plane_idx in plane_indices)

# Combine multiple plane data
#FBS.CombinePlanes(SaveFolder,ops0)
FBS.CombinePlanes(SaveFolder,ops0)

range(0, 3)
Processing plane 0
Saving folder already exists.
Binning movie in chunks of length 06
Binned movie of size [1366,512,512] created in 10.38 sec.
Binned movie denoised (for cell detection only) in 32.47 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.14, threshold 11.38 
0 ROIs, score=100.37
Detected 120 ROIs, 24.43 sec
After removing overlaps, 119 ROIs remain
added enhanced mean image
Masks created, 0.66 sec.
Extracted fluorescence from 119 ROIs in 8200 frames, 10.45 sec.
Extracted fluorescence from 119 ROIs in 8200 frames, 10.45 sec.
['skew', 'npix_norm', 'compact']
Processing plane 1
Binning movie in chunks of length 06
Binned movie of size [1366,512,512] created in 8.30 sec.
Binned movie denoised (for cell detection only) in 31.36 sec.
NOTE: FORCED spatial scale ~12 pixels, time epochs 1.14, threshold 11.38 
0 ROIs, score=77.60
Detected 109 ROIs, 24.14 sec
After removing overlaps, 109 ROIs remain
added enhanced mean image
Masks created, 0.63 sec.
Extracted fluor

Run Suite2p gui from command line, load combined data above, do necessary mannual correction. Saved the data.

Once above mannual correction is done; update the result in each plane

In [10]:
FBS.PostMannual(SaveFolder,ops0)

339 of units including 118 identified cells in combined planes
